In [24]:
import pandas as pd
import numpy as np

In [25]:
import utils; reload(utils)
from utils import *

In [26]:
path = 'data/ml-20m/'
rating_file_path = path + 'ratings.csv'

In [27]:
rating_df = pd.read_csv(rating_file_path)

In [28]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [29]:
len(rating_df)

20000263

In [46]:
rating_df = rating_df[:10000]

In [47]:
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()

In [48]:
users = rating_df.userId.unique()
movies = rating_df.movieId.unique()

In [49]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [101]:
rating_df.movieId = rating_df.movieId.apply(lambda x: movieid2idx[x])
rating_df.userId = rating_df.userId.apply(lambda x: userid2idx[x])

In [103]:
# rating_df

In [51]:
user_min, user_max, movie_min, movie_max = (rating_df.userId.min(), 
    rating_df.userId.max(), rating_df.movieId.min(), rating_df.movieId.max())
user_min, user_max, movie_min, movie_max

(0, 10, 0, 697)

In [52]:
n_users = rating_df.userId.nunique()
n_movies = rating_df.movieId.nunique()
n_users, n_movies

(11, 698)

In [53]:
n_factors = 50
np.random.seed = 42

In [54]:
msk = np.random.rand(len(rating_df))<0.8

In [55]:
train_df = rating_df[msk]
val_df = rating_df[~msk]

In [56]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [57]:
x = merge([u,m], mode='dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [58]:
model.fit([train_df.userId, train_df.movieId], train_df.rating, batch_size=64, nb_epoch=1, 
          validation_data=([val_df.userId, val_df.movieId], val_df.rating))

Train on 793 samples, validate on 207 samples
Epoch 1/1
793/793 [==============================] - 0s - loss: 14.6097 - val_loss: 15.2912


In [59]:
model.optimizer.lr=0.01

In [62]:
model.fit([train_df.userId, train_df.movieId], train_df.rating, batch_size=64, nb_epoch=3, 
          validation_data=([val_df.userId, val_df.movieId], val_df.rating))

Train on 793 samples, validate on 207 samples
Epoch 1/3
793/793 [==============================] - 0s - loss: 14.3664 - val_loss: 15.2774
Epoch 2/3
793/793 [==============================] - 0s - loss: 14.2250 - val_loss: 15.2635
Epoch 3/3
793/793 [==============================] - 0s - loss: 14.0304 - val_loss: 15.2433


In [45]:
model.optimizer.lr=0.001


In [63]:
model.fit([train_df.userId, train_df.movieId], train_df.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val_df.userId, val_df.movieId], val_df.rating))

Train on 793 samples, validate on 207 samples
Epoch 1/6
793/793 [==============================] - 0s - loss: 13.7762 - val_loss: 15.2147
Epoch 2/6
793/793 [==============================] - 0s - loss: 13.4606 - val_loss: 15.1784
Epoch 3/6
793/793 [==============================] - 0s - loss: 13.0813 - val_loss: 15.1335
Epoch 4/6
793/793 [==============================] - 0s - loss: 12.6435 - val_loss: 15.0783
Epoch 5/6
793/793 [==============================] - 0s - loss: 12.1474 - val_loss: 15.0136

In [64]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [65]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [66]:
def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

In [67]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [68]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, mb], mode='sum')
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [70]:
model.fit([train_df.userId, train_df.movieId], train_df.rating, batch_size=64, nb_epoch=1, 
          validation_data=([val_df.userId, val_df.movieId], val_df.rating))

Train on 793 samples, validate on 207 samples
Epoch 1/1
793/793 [==============================] - 0s - loss: 14.6857 - val_loss: 15.3561


In [71]:
model.optimizer.lr=0.01


In [75]:
model.fit([train_df.userId, train_df.movieId], train_df.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val_df.userId, val_df.movieId], val_df.rating))


Train on 793 samples, validate on 207 samples
Epoch 1/6
793/793 [==============================] - 0s - loss: 14.5274 - val_loss: 15.2497
Epoch 2/6
793/793 [==============================] - 0s - loss: 14.3657 - val_loss: 15.1440
Epoch 3/6
793/793 [==============================] - 0s - loss: 14.1857 - val_loss: 15.0356
Epoch 4/6
793/793 [==============================] - 0s - loss: 13.9746 - val_loss: 14.9256
Epoch 5/6
793/793 [==============================] - 0s - loss: 13.7241 - val_loss: 14.8116

In [76]:
model.optimizer.lr=0.001

In [78]:
model.fit([train_df.userId, train_df.movieId], train_df.rating, batch_size=64, nb_epoch=10, 
          validation_data=([val_df.userId, val_df.movieId], val_df.rating))

Train on 793 samples, validate on 207 samples
Epoch 1/10
793/793 [==============================] - 0s - loss: 13.0718 - val_loss: 14.5706
Epoch 2/10
793/793 [==============================] - 0s - loss: 12.6629 - val_loss: 14.4429
Epoch 3/10
793/793 [==============================] - 0s - loss: 12.1990 - val_loss: 14.3099
Epoch 4/10
793/793 [==============================] - 0s - loss: 11.6858 - val_loss: 14.1715
Epoch 5/10
793/793 [==============================] - 0s - loss: 11.1248 - val_loss: 14.0257

In [79]:
model.fit([train_df.userId, train_df.movieId], train_df.rating, batch_size=64, nb_epoch=5, 
          validation_data=([val_df.userId, val_df.movieId], val_df.rating))

Train on 793 samples, validate on 207 samples
Epoch 1/5
793/793 [==============================] - 0s - loss: 7.2293 - val_loss: 13.0759
Epoch 2/5
793/793 [==============================] - 0s - loss: 6.5785 - val_loss: 12.9118
Epoch 3/5
793/793 [==============================] - 0s - loss: 5.9414 - val_loss: 12.7505
Epoch 4/5
793/793 [==============================] - 0s - loss: 5.3363 - val_loss: 12.5881
Epoch 5/5
793/793 [==============================] - 0s - loss: 4.7591 - val_loss: 12.4308

In [81]:
# model.save_weights(model_path+'bias.h5')
# model.load_weights(model_path+'bias.h5')


In [82]:
model.predict([np.array([3]), np.array([6])])


array([[ 0.5782]], dtype=float32)

In [85]:
g=rating_df.groupby('movieId')['rating'].count()
topMovies=g.sort_values(ascending=False)[:20]
topMovies = np.array(topMovies.index)

In [86]:
get_movie_bias = Model(movie_in, mb)
movie_bias = get_movie_bias.predict(topMovies)
movie_ratings = [(b[0], movie_names[movies[i]]) for i,b in zip(topMovies,movie_bias)]

In [87]:
sorted(movie_ratings, key=itemgetter(0))[:15]


[(0.069590956, 'Sense and Sensibility (1995)'),
 (0.073112294, 'Reality Bites (1994)'),
 (0.088911332, 'Ed Wood (1994)'),
 (0.10526828, 'Kids (1995)'),
 (0.10796748, 'Love & Human Remains (1993)'),
 (0.11177368, 'Sudden Death (1995)'),
 (0.12254011, 'Nixon (1995)'),
 (0.1333552, 'Red Rock West (1992)'),
 (0.14124554, 'Drop Zone (1994)'),
 (0.14764997, 'Nine Months (1995)'),
 (0.15012211, 'Dangerous Minds (1995)'),
 (0.15061015, 'Mrs. Parker and the Vicious Circle (1994)'),
 (0.15061492, 'Fall Time (1995)'),
 (0.15632848, 'Exotica (1994)'),
 (0.15642078, 'Money Train (1995)')]

In [88]:
sorted(movie_ratings, key=itemgetter(0), reverse=True)[:15]


[(0.19729555, 'Casino (1995)'),
 (0.17779985, 'Moonlight and Valentino (1995)'),
 (0.17387779, 'Now and Then (1995)'),
 (0.17345433, 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)'),
 (0.16444403, 'To Die For (1995)'),
 (0.15642078, 'Money Train (1995)'),
 (0.15632848, 'Exotica (1994)'),
 (0.15061492, 'Fall Time (1995)'),
 (0.15061015, 'Mrs. Parker and the Vicious Circle (1994)'),
 (0.15012211, 'Dangerous Minds (1995)'),
 (0.14764997, 'Nine Months (1995)'),
 (0.14124554, 'Drop Zone (1994)'),
 (0.1333552, 'Red Rock West (1992)'),
 (0.12254011, 'Nixon (1995)'),
 (0.11177368, 'Sudden Death (1995)')]

In [89]:
get_movie_emb = Model(movie_in, m)
movie_emb = np.squeeze(get_movie_emb.predict([topMovies]))
movie_emb.shape

(20, 50)

In [90]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
movie_pca = pca.fit(movie_emb.T).components_

In [91]:
fac0 = movie_pca[0]


In [92]:
movie_comp = [(f, movie_names[movies[i]]) for f,i in zip(fac0, topMovies)]


In [93]:
sorted(movie_comp, key=itemgetter(0), reverse=True)[:10]


[(-0.15131608587477641, 'To Die For (1995)'),
 (-0.1531134758216495, 'Ed Wood (1994)'),
 (-0.18424551241724602, 'Nine Months (1995)'),
 (-0.18706022179633669, 'Exotica (1994)'),
 (-0.18929161935597039, 'Sense and Sensibility (1995)'),
 (-0.19394715205792651, 'Money Train (1995)'),
 (-0.20787907801021491, 'Reality Bites (1994)'),
 (-0.21503742006118601, 'Nixon (1995)'),
 (-0.21892849278006771, 'Dangerous Minds (1995)'),
 (-0.22081260854090085, 'Fall Time (1995)')]

In [94]:
sorted(movie_comp, key=itemgetter(0))[:10]


[(-0.27128168373205008, 'Casino (1995)'),
 (-0.26626978386615263, 'Sudden Death (1995)'),
 (-0.25892865669991355, 'Drop Zone (1994)'),
 (-0.25587561735009445,
  'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)'),
 (-0.25299419524790012, 'Red Rock West (1992)'),
 (-0.24774472359267968, 'Moonlight and Valentino (1995)'),
 (-0.24104164503817954, 'Mrs. Parker and the Vicious Circle (1994)'),
 (-0.23766448526882608, 'Kids (1995)'),
 (-0.23444456833886437, 'Now and Then (1995)'),
 (-0.23062801194944108, 'Love & Human Remains (1993)')]

In [95]:
fac1 = movie_pca[1]


In [96]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [97]:
x = merge([u, m], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')

In [98]:
nn.fit([train_df.userId, train_df.movieId], train_df.rating, batch_size=64, nb_epoch=8, 
          validation_data=([val_df.userId, val_df.movieId], val_df.rating))

Train on 793 samples, validate on 207 samples
Epoch 1/8
793/793 [==============================] - 0s - loss: 14.2710 - val_loss: 14.2843
Epoch 2/8
793/793 [==============================] - 0s - loss: 13.0870 - val_loss: 12.9025
Epoch 3/8
793/793 [==============================] - 0s - loss: 11.4653 - val_loss: 10.7630
Epoch 4/8
793/793 [==============================] - 0s - loss: 9.1091 - val_loss: 7.7531
Epoch 5/8
793/793 [==============================] - 0s - loss: 6.1816 - val_loss: 4.3623

In [100]:
nn.predict([np.array([3]), np.array([6])])

array([[ 2.857]], dtype=float32)